In [31]:
# %pip install locationtagger
# %pip install nltk
# %pip install spacy
# %pip install lxml_html_clean

In [1]:
# !python -m spacy download en_core_web_sm

In [2]:
# import spacy
# nlp = spacy.load("en_core_web_sm")
# print("Model loaded successfully!")

In [3]:
# import nltk

 
# # essential entity models downloads
# nltk.downloader.download('maxent_ne_chunker')
# nltk.downloader.download('words')
# nltk.downloader.download('treebank')
# nltk.downloader.download('maxent_treebank_pos_tagger')
# nltk.downloader.download('punkt')
# nltk.download('averaged_perceptron_tagger')

# New Section

In [4]:
# from google.colab import drive
# drive.mount('/content/MyDrive')

In [5]:
import numpy as np
import pandas as pd
import locationtagger

In [6]:
raw_df = pd.read_csv(r'D:\DINESH_DATA_SCIENCE\MY_PROJECT\REAL ESTATE PRICE PREDICTION\Real Estate Data V21.csv')

In [7]:
print(list(raw_df.columns))

['Name', 'Property Title', 'Price', 'Location', 'Total_Area', 'Price_per_SQFT', 'Description', 'Baths', 'Balcony']


In [8]:
raw_df = raw_df.drop(columns=['Description'], axis=1)

In [9]:
def extract_entities_lst(line):
    entities_lst = []
    if ',' in line:
        entities_lst = line.split(',')
    if entities_lst:
        entities_lst = [entity.rstrip(' ') for entity in entities_lst]
        entities_lst = [entity.lstrip(' ') for entity in entities_lst]
    return entities_lst
    
def extract_entities_from_text(sample_text):
    entities = extract_entities_lst(sample_text)
    # extracting entities.
    place_entity = locationtagger.find_locations(text = sample_text)
    
    country_str = ''
    # getting all countries
    if place_entity.countries:
        country_str = list(place_entity.countries)[0]
    elif place_entity.other_countries:
        country_str = list(place_entity.other_countries)[0]
    if country_str and country_str in entities:
        entities.remove(country_str)


    state_str = ''
    # getting all states
    if place_entity.regions:
        state_str = list(place_entity.regions)[0]
    elif place_entity.other_regions:
        state_str = list(place_entity.other_regions)[0]
    if state_str and state_str in entities:
        entities.remove(state_str)


    city_str = ''
    # getting all cities
    if place_entity.cities:
        city_str = list(place_entity.cities)[0]
    if city_str and city_str in entities:
            entities.remove(city_str)
        
    # entities_str = ', '.join(entities)
    return entities, city_str, state_str, country_str

In [10]:
state_lst = []
country_lst = []
city_lst = []
other_lst = []
named_lst = []

In [11]:
limiter = ' for sale in '
new_property_title_lst = []
for i in range(len(raw_df['Location'])):
    # print("Current Instance: ", i)
    new_property_title_lst.append(raw_df['Property Title'][i].split(limiter)[0])
    named_entities = extract_entities_lst(raw_df['Name'][i])
    given_loc_entities = extract_entities_lst(raw_df['Location'][i])
    prop_loc_entities = extract_entities_lst(raw_df['Property Title'][i].split(limiter)[1])
    for prop_loc in prop_loc_entities:
        if prop_loc not in given_loc_entities:
            given_loc_entities.append(prop_loc)
    
    given_loc_str = ', '.join(given_loc_entities)
    for given_loc in given_loc_entities:
        if given_loc in named_entities:
            named_entities.remove(given_loc)
    named_entities_str = ', '.join(named_entities)
    
    loc_mark_in_city, loc_city, loc_state, loc_country = extract_entities_from_text(given_loc_str)
    
    if loc_mark_in_city:
        other_lst.append(loc_mark_in_city)
    else:
        other_lst.append(np.nan)
        
    if loc_city:
        city_lst.append(loc_city)
    else:
        city_lst.append(np.nan)
    
    if loc_state:
        state_lst.append(loc_state)
    else:
        state_lst.append(np.nan)
    
    if loc_country:
        country_lst.append(loc_country)
    else:
        country_lst.append(np.nan)
    
    if not named_entities_str:
        named_entities_str = np.nan
    named_lst.append(named_entities_str)

In [12]:
raw_df['Name'] = named_lst
raw_df['Property Title'] = new_property_title_lst
raw_df['Mark_in_City'] = other_lst
raw_df['City'] = city_lst
raw_df['State'] = state_lst
raw_df['Country'] = country_lst

* Extracting number of bedrooms Type

In [13]:
property_title_lst = list(raw_df['Property Title'])
bedroom_lst = list()

for i in range(len(property_title_lst)):
    try:
        if 'bhk' in property_title_lst[i].lower():
            bedroom_lst.append(property_title_lst[i].lower().split(' bhk')[0])
        else:
            bedroom_lst.append(str(1))
    except Exception as e:
        print('Exception: ', str(e))
        bedroom_lst.append(str(1))

raw_df['Bedrooms'] = bedroom_lst

In [14]:
raw_df['Bedrooms'] = raw_df['Bedrooms'].mask(raw_df['Bedrooms'] == '5+', float(5.5))

In [15]:
unique_lst = list(raw_df['Bedrooms'].unique())
print(unique_lst)
for unique_ele in unique_lst:
    if isinstance(unique_ele, str):
        raw_df['Bedrooms'] = raw_df['Bedrooms'].mask(raw_df['Bedrooms'] == unique_ele, float(unique_ele))

['4', '10', '3', '7', '2', '1', '6', '5', '8', '1.5', '2.5', '9', '3.5', 5.5, '10.5', '4.5', '9.5', '8.5', '7.5']


In [16]:
print(raw_df['Bedrooms'].unique())

[4.0 10.0 3.0 7.0 2.0 1.0 6.0 5.0 8.0 1.5 2.5 9.0 3.5 5.5 10.5 4.5 9.5 8.5
 7.5]


* Extracting House/Flat/Villa Type

In [17]:
house_flat_lst = []
for i in range(len(property_title_lst)):
    if 'flat' in property_title_lst[i].lower():
        house_flat_lst.append('flat')
    elif 'house' in property_title_lst[i].lower():
        house_flat_lst.append('house')
    elif 'villa' in property_title_lst[i].lower():
        house_flat_lst.append('villa')
    else:
        house_flat_lst.append('other')
        print('Other: ', property_title_lst[i])
        
raw_df['House/Flat/Villa'] = house_flat_lst

In [18]:
print(len(new_property_title_lst))
new_property_title_lst_ = []
hall_lst = []
kitchen_lst = []
studio_shop_lst = []
new_property_type_lst_ = []
for new_property_title in new_property_title_lst:
    temp_hall = 0
    temp_kitchen = 0
    temp_studio = 0
    if 'BHK' in new_property_title:
        new_property_title_lst_.append(new_property_title.split('BHK')[1].strip())
        new_property_type_lst_.append('BHK')
        temp_hall = 1
        temp_kitchen = 1
    elif 'RK' in new_property_title:
        new_property_title_lst_.append(new_property_title.split('RK')[1].strip())
        new_property_type_lst_.append('RK')
        temp_kitchen = 1
    elif 'Studio' in new_property_title:
        new_property_title_lst_.append(new_property_title.split('Studio')[1].strip())
        new_property_type_lst_.append('Studio')
        temp_studio = 1
    elif 'Shop' in new_property_title:
        new_property_title_lst_.append(new_property_title.split('Shop')[1].strip())
        new_property_type_lst_.append('Shop')
        temp_studio = 1
    elif 'BH' in new_property_title:
        new_property_title_lst_.append(new_property_title.split('BH')[1].strip())
        new_property_type_lst_.append('BH')
        temp_hall = 1
    elif 'R' in new_property_title:
        new_property_title_lst_.append(new_property_title.split('R')[1].strip())
        new_property_type_lst_.append('R')
    else:
        new_property_title_lst_.append(new_property_title)
        new_property_type_lst_.append('Other')
    hall_lst.append(temp_hall)
    kitchen_lst.append(temp_kitchen)
    studio_shop_lst.append(temp_studio)
# Print the cleaned list
print(len(new_property_title_lst_))

14528
14528


In [19]:
raw_df['Property_Type'] = new_property_type_lst_
raw_df['Property Title'] = new_property_title_lst_
raw_df['Hall'] = hall_lst
raw_df['Kitchen'] = kitchen_lst
raw_df['Studio/Shop'] = studio_shop_lst

In [20]:
new_property_title_lst = []
for i in range(len(raw_df['Property Title'])):
        new_property_title_lst.append(raw_df['Property Title'][i] + ' ' + str(i))
raw_df['Property Title'] = new_property_title_lst

* data cleaning the Target Variable (Price in K)

In [21]:
price_lst = list(raw_df['Price'])
price_lst_ = []
for price in price_lst:
    try:
        price_lst_.append(price.replace('₹', ''))
    except Exception as e:
        print('Exception: ', str(e))
        price_lst_.append(price)
price_lst = price_lst_

In [22]:
price_lst_inr = []
for current_price in price_lst:
    try:
        if 'Cr' in current_price:
            price_lst_inr.append(int(float(current_price.split('Cr')[0]) * 100) * 100)
        elif 'L' in current_price:
            price_lst_inr.append(int(float(current_price.split('L')[0]) * 100))
        elif 'K' in current_price:
            price_lst_inr.append(int(float(current_price.split('K')[0]) * 100) // 100)
        else:
            if 'k' in current_price:
                price_lst_inr.append(int(float(current_price.split('k')[0]) * 100) // 100)
            else:
                price_lst_inr.append(np.nan)
    except Exception as e:
        print('Exception: ', str(e))
        print("Error")
        print(current_price)

In [23]:
raw_df['Price (in Thousands)'] = price_lst_inr

In [24]:
raw_df['Price (in Thousands)'] = raw_df['Price (in Thousands)'].bfill()

In [25]:
print(list(raw_df.columns))

['Name', 'Property Title', 'Price', 'Location', 'Total_Area', 'Price_per_SQFT', 'Baths', 'Balcony', 'Mark_in_City', 'City', 'State', 'Country', 'Bedrooms', 'House/Flat/Villa', 'Property_Type', 'Hall', 'Kitchen', 'Studio/Shop', 'Price (in Thousands)']


In [26]:
raw_df = raw_df.drop(columns=['Location', 'Price'], axis=1)

In [27]:
city_lst = []
state_lst = []
for i in range(len(raw_df['Mark_in_City'])):
    if isinstance(raw_df['City'][i], float) or isinstance(raw_df['State'][i], float):
        if 'New Delhi' in raw_df['Mark_in_City'][i]:
            city_lst.append('New Delhi')
            state_lst.append('National Union Territory')
        else:
            print('Mark_in_City : ', raw_df['Mark_in_City'][i])
            city_lst.append(raw_df['City'][i])
            state_lst.append(raw_df['State'][i])
    else:
        city_lst.append(raw_df['City'][i])
        state_lst.append(raw_df['State'][i])

raw_df['City'] = city_lst
raw_df['State'] = state_lst


In [28]:
raw_df['Country'] = raw_df['Country'].fillna('India')

In [29]:
print(raw_df.isnull().any())

Name                     True
Property Title          False
Total_Area              False
Price_per_SQFT          False
Baths                   False
Balcony                 False
Mark_in_City            False
City                    False
State                   False
Country                 False
Bedrooms                False
House/Flat/Villa        False
Property_Type           False
Hall                    False
Kitchen                 False
Studio/Shop             False
Price (in Thousands)    False
dtype: bool


In [30]:
raw_df.to_excel('D:\DINESH_DATA_SCIENCE\MY_PROJECT\REAL ESTATE PRICE PREDICTION\Real_Estate_Data_V21.xlsx', index=False)